In [5]:
import mne
from mne.io import read_raw_eeglab

from pycrostates.cluster import ModKMeans
from pycrostates.datasets import lemon


# load sample dataset
raw_fname = lemon.data_path(subject_id='010017', condition='EC')
raw = read_raw_eeglab(raw_fname, preload=True)
raw.crop(0, 10)  # crop the dataset to speed up computation
raw.pick('eeg')  # select EEG channels
#raw.set_eeg_reference('average')  # Apply a common average reference





Reading C:\Users\Victor\pycrostates_data\PREPROCESSED_LEMON\sub-010017_EC.fdt
Reading 0 ... 119451  =      0.000 ...   477.804 secs...


C:\Users\Victor\AppData\Local\Temp\ipykernel_2660\1868782884.py:10: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = read_raw_eeglab(raw_fname, preload=True)
C:\Users\Victor\AppData\Local\Temp\ipykernel_2660\1868782884.py:10: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = read_raw_eeglab(raw_fname, preload=True)


Measurement date,Unknown
Experimenter,Unknown
Participant,Unknown
Digitized points,64 points
Good channels,61 EEG
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,250.00 Hz
Highpass,0.00 Hz
Lowpass,125.00 Hz


In [ ]:
epochs = mne.make_fixed_length_epochs(raw, duration=1.0, preload=True)

n_clusters = 5
ModK = ModKMeans(n_clusters=n_clusters, random_state=42)
ModK.fit(epochs, n_jobs=5)

# We need to make sure to reject edges as we are working with epochs, so we don't want to take into account transition happening between 2 epochs
segmentation = ModK.predict(epochs, reject_edges=True) #

In [45]:
import numpy as np
import scipy.signal
from pycrostates.segmentation.transitions import compute_transition_matrix

...

n_clusters = 5
ModK = ModKMeans(n_clusters=n_clusters, random_state=42)
ModK.fit(epochs, n_jobs=5)

labels = []
epochs_data = epochs.get_data()
n_sample_per_epoch = epochs_data.shape[2]

for e, epoch in enumerate(epochs_data):
    epoch_gfp = np.std(epoch, axis=0) # compute gfp
    epoch_peaks = scipy.signal.find_peaks(epoch_gfp, distance=3)[0] # find peak location (in the current epoch)
    epoch_peaks += e * n_sample_per_epoch # change the peak location to the global index
    epoch_peaks_labels = ModK.labels_[epoch_peaks].astype(int) # get the label of the peaks
    labels.extend(epoch_peaks_labels)
    labels.append(-1) # add -1 at the end of each epoch to make sure compute_transition_matrix ignore transition between epochs

labels = np.array(labels)
compute_transition_matrix(labels, ModK.n_clusters)

C:\Users\Victor\AppData\Local\Temp\ipykernel_3492\3786942813.py:13: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  epochs_data = epochs.get_data()


array([[0.        , 0.44444444, 0.18518519, 0.22222222, 0.14814815],
       [0.3442623 , 0.        , 0.19672131, 0.19672131, 0.26229508],
       [0.15      , 0.425     , 0.        , 0.15      , 0.275     ],
       [0.325     , 0.1       , 0.225     , 0.        , 0.35      ],
       [0.3125    , 0.27083333, 0.20833333, 0.20833333, 0.        ]])

In [51]:
n_clusters = 5
ModK = ModKMeans(n_clusters=n_clusters, random_state=42)
ModK.fit(epochs, n_jobs=5)


peaks_index, epochs_index = [], []
epochs_data = epochs.get_data()
n_sample_per_epoch = epochs_data.shape[2]

for e, epoch in enumerate(epochs_data):
    epoch_gfp = np.std(epoch, axis=0) # compute gfp
    epoch_peaks = scipy.signal.find_peaks(epoch_gfp, distance=3)[0] # find peak location (in the current epoch)
    epoch_peaks += e * n_sample_per_epoch # change the peak location to the global index
    peaks_index.append(epoch_peaks)
    epochs_index.append(e)

peak_index = [item for row in matrix for item in row]
gfp_peaks_data = np.hstack(epochs_data)[:, peaks_index.flatten()]
raw_gfp = mne.io.RawArray(gfp_peaks_data, epochs.info)


C:\Users\Victor\AppData\Local\Temp\ipykernel_3492\1733787885.py:7: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  epochs_data = epochs.get_data()


AttributeError: 'list' object has no attribute 'flatten'

In [9]:
import numpy as np

def pairwise_cosine_similarity(A, B):
    # Compute dot product of all pairs of vectors between A and B
    dot_products = np.sum(A * B, axis=1)
    
    # Compute norms of vectors in A and B
    norm_A = np.linalg.norm(A, axis=1)
    norm_B = np.linalg.norm(B, axis=1)
    
    # Compute pairwise cosine similarity
    similarities = dot_products / (norm_A * norm_B)
    
    return similarities

# Example usage:
# Create two matrices A and B
data = raw.get_data()
A = data[:, :10]  # Example matrix A
B = data[:, 10:20]# Example matrix B

A[0] = 1
# Compute pairwise cosine similarity between corresponding pairs of vectors from A and B
similarities = pairwise_cosine_similarity(A, B)

print("Pairwise cosine similarities:")
print(similarities)

Pairwise cosine similarities:
[-0.88960069 -0.52650275 -0.44525636 -0.50539258 -0.21907496  0.1474366
 -0.48859197 -0.39191663  0.017335    0.30847924  0.05141247 -0.46169683
  0.10246357  0.51774697  0.71619703 -0.34952813 -0.59010702  0.31178774
  0.82140473  0.17823281 -0.6207871  -0.91049516 -0.82097712  0.32606955
  0.36015225 -0.66678265 -0.93639654 -0.86016585 -0.65252435 -0.62119827
 -0.85428709 -0.71636988 -0.83021528 -0.36011766 -0.26945252 -0.42333924
 -0.59915667  0.24471488 -0.09734654 -0.40165124 -0.20802687  0.10455659
 -0.45169751 -0.24339344  0.25451128  0.81244125  0.14459244 -0.67411437
 -0.34308167  0.4091359   0.75020719 -0.53287629 -0.89727877 -0.40925861
  0.70889051 -0.27092024 -0.96432043 -0.8853609   0.19048041 -0.09020065
 -0.70621667]


In [31]:
from pycrostates.preprocessing import extract_gfp_peaks


n_clusters = 5
ModK = ModKMeans(n_clusters=n_clusters, random_state=42)


peaks = extract_gfp_peaks(epochs)
ModK.fit(peaks, n_jobs=5)

data = np.hstack(epochs.get_data())
gfp = np.std(data, axis=0)
peaks = scipy.signal.find_peaks(gfp)[0]
labels = ModK.labels_[peaks].astype(int)

compute_transition_matrix(labels, ModK.n_clusters)


(314,)

array([1, 2, 1, 4, 4, 1, 1, 2, 1, 0, 1, 2, 1, 0, 1, 1, 3, 0, 0, 1, 4, 0,
       1, 4, 3, 2, 1, 4, 1, 4, 4, 0, 3, 3, 0, 0, 0, 0, 4, 0, 1, 0, 1, 0,
       4, 2, 2, 2, 3, 3, 3, 2, 1, 4, 0, 1, 0, 1, 2, 4, 1, 4, 1, 3, 4, 1,
       4, 2, 4, 1, 4, 2, 4, 3, 4, 0, 2, 4, 4, 2, 4, 1, 1, 3, 2, 0, 2, 0,
       3, 4, 2, 4, 4, 4, 3, 1, 4, 2, 1, 3, 3, 3, 2, 1, 1, 0, 1, 0, 0, 2,
       1, 3, 0, 1, 1, 0, 1, 0, 2, 0, 4, 0, 1, 2, 3, 0, 3, 4, 0, 1, 0, 3,
       4, 0, 2, 3, 3, 4, 4, 4, 2, 4, 1, 1, 3, 4, 3, 4, 0, 1, 4, 3, 2, 3,
       3, 0, 2, 0, 0, 3, 0, 3, 0, 3, 2, 4, 3, 0, 3, 0, 4, 0, 0, 1, 0, 1,
       0, 2, 4, 0, 0, 4, 0, 1, 0, 0, 2, 1, 4, 4, 4, 3, 4, 1, 1, 1, 2, 1,
       1, 4, 3, 4, 2, 1, 1, 2, 1, 1, 4, 2, 1, 1, 2, 1, 2, 3, 3, 2, 2, 0,
       1, 1, 3, 2, 1, 3, 0, 1, 0, 4, 3, 3, 1, 4, 4, 1, 2, 3, 4, 1, 2, 2,
       1, 1, 1, 4, 1, 4, 0, 1, 0, 0, 1, 3, 0, 1, 3, 0, 4, 4, 1, 1, 1, 0,
       4, 3, 3, 0, 0, 1, 0, 3, 4, 0, 0, 1, 0, 2, 1, 0, 3, 2, 1, 3, 2, 4,
       4, 1, 3, 1, 0, 3, 0, 2, 0, 1, 0, 0, 0, 0, 1,